In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%writefile task.py
def p(g):
  
    first_mapping = {5: 0, 3: 1, 7: 2}  
    
   
    new_first = first_mapping.get(g[0][0], 0)
    
   
    return [[new_first]] + [g[3]] + [g[4]] + [g[2]] + [g[3]] + [g[4]]

#

In [ ]:
import json

# Path to the ARC/Code Golf task file
task_path = "/kaggle/input/google-code-golf-2025/task001.json"

# Load the JSON data
with open(task_path, "r") as f:
    task_data = json.load(f)

# Get the first training example
first_train_example = task_data["train"][0]

# Display the input grid
print("First training example input:")
for row in first_train_example["input"]:
    print(row)

# Display the output grid
print("\nFirst training example output:")
for row in first_train_example["output"]:
    print(row)


In [ ]:
def p(g):
 
    row = g[0]
 
    new_row = row[:3] + [0] + row[3:6] + [0] + row[6:]
    return [new_row]

In [ ]:
def p(g):
    
    if len(g) >= 5:
        return [[0]] + [g[3]] + [g[4]] + [g[2]] + [g[3]] + [g[4]]
    else:
        
        if len(g) == 1 and len(g[0]) == 8:
            return [[0,0,0,0,7,7,0,7,7]]
        else:
            new_row = g[0][:3] + [0] + g[0][3:6] + [0] + g[0][6:]
            return [new_row]

In [ ]:
def p(g):
  
    row = g[0]
    row = row[:-1]   
    row.insert(3, 0)
    row.insert(6, 0)
    return [row]

In [ ]:
def transform_grid(input_grid):
    """Transform the grid according to the observed pattern"""
    output = []
    for row in input_grid:
       
        row0 = []
        row1 = []
        row2 = []
        for val in row:
            # Pattern for each value
            row0.extend([0, val, val])
            row1.extend([val, val, val])
            row2.extend([0, val, val])
        output.append(row0)
        output.append(row1)
        output.append(row2)
    return output


SOLUTION_CODE = "p=lambda g:sum([[[0,x,x]for x in r]+[[x,x,x]for x in r]+[[0,x,x]for x in r]for r in g],[])"

In [ ]:
import json
import importlib.util
import numpy as np
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile

def load_task_data(task_num):
    """Load task JSON data with proper padding"""
    path = f"/kaggle/input/google-code-golf-2025/task{task_num:03d}.json"
    with open(path) as f:
        return json.load(f)

def visualize_grid(grid, title="Grid", figsize=(4,4)):
    """Visualize grid with correct ARC color scheme"""
    colors = [
        (0, 0, 0), (30, 147, 255), (250, 61, 49), (78, 204, 48),
        (255, 221, 0), (153, 153, 153), (229, 59, 163),
        (255, 133, 28), (136, 216, 241), (147, 17, 49)
    ]
    img = np.zeros((len(grid), len(grid[0])), dtype=int)  # ✅ fixed parentheses
    for r, row in enumerate(grid):
        for c, val in enumerate(row):
            img[r, c] = val
    plt.figure(figsize=figsize)
    plt.imshow(img, cmap='viridis', vmin=0, vmax=9)
    plt.title(title)
    plt.axis('off')
    plt.colorbar()
    plt.show()

def verify_solution(task_num, solution_code):
    """Complete verification system"""
    task_data = load_task_data(task_num)
    all_examples = task_data["train"] + task_data["test"] + task_data["arc-gen"]
    
  
    with open("solution.py", "w") as f:
        f.write(solution_code)
    
    # Import solution
    try:
        spec = importlib.util.spec_from_file_location("solution", "solution.py")
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        solution_func = module.p
    except Exception as e:
        print(f"Error loading solution: {str(e)}")
        return {"passed": 0, "failed": [{"error": str(e)}]}
    
    results = {"passed": 0, "failed": []}
    
    for idx, example in enumerate(all_examples):
        input_grid = [row[:] for row in example["input"]]
        expected = example["output"]
        
        try:
            output = solution_func(input_grid)
        except Exception as e:
            results["failed"].append({
                "idx": idx, "type": "exception", "error": str(e),
                "input": input_grid, "expected": expected
            })
            continue
            
        if output == expected:
            results["passed"] += 1
        else:
            results["failed"].append({
                "idx": idx, "type": "mismatch",
                "input": input_grid,
                "output": output,
                "expected": expected
            })
    
    # Print summary
    total = len(all_examples)
    print(f"Task {task_num:03d} Verification Results:")
    print(f"✅ Passed: {results['passed']}/{total}")
    print(f"❌ Failed: {len(results['failed'])}/{total}")
    
    # Show first failure
    if results["failed"]:
        first_fail = results["failed"][0]
        print(f"\nFirst Failure (Example Index: {first_fail['idx']}):")
        
        print("Input Grid:")
        for row in first_fail["input"]:
            print(row)
        visualize_grid(first_fail["input"], "Input Grid")
        
        if first_fail["type"] == "exception":
            print(f"🚨 Error: {first_fail['error']}")
        else:
            print("\nYour Output:")
            for row in first_fail["output"]:
                print(row)
            visualize_grid(first_fail["output"], "Your Output")
            
            print("\nExpected Output:")
            for row in first_fail["expected"]:
                print(row)
            visualize_grid(first_fail["expected"], "Expected Output")
    
    return results


SOLUTION_CODE = """
p = lambda g: [
    [z for x in r for z in ([x]*3 if i==1 else [0, x, x])]
    for r in g
    for i in range(3)
]
""".strip()

print("Verifying Task 001 Solution...")
results = verify_solution(1, SOLUTION_CODE)

if not results.get("failed"):
    with open("task001.py", "w") as f:
        f.write(SOLUTION_CODE)
    print("\n Solution verified! Saved to task001.py")
    print(f"Solution size: {len(SOLUTION_CODE)} bytes")
    
    print("\nSolution code:")
    print(SOLUTION_CODE)
else:
    print("\n Solution needs improvement")


def create_submission():
    """Package solutions into submission.zip"""
    solution_files = [f for f in os.listdir() if f.startswith("task") and f.endswith(".py")]
    
    if not solution_files:
        print("No solution files found for submission")
        return
    
    with ZipFile("submission.zip", "w") as zipf:
        for file in solution_files:
            zipf.write(file)
    
    print(f"\n Created submission.zip with {len(solution_files)} solutions")

create_submission()
